In [ ]:
# The dataset used in this project was downloaded from http://insideairbnb.com.
# using pandas to read in the dataset
import pandas as pd
df = pd.read_csv('/content/Ash_listing.csv') 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

In [143]:
#check the dataframe
df.info()
df.index
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009 entries, 0 to 3008
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            3009 non-null   int64  
 1   listing_url                                   3009 non-null   object 
 2   scrape_id                                     3009 non-null   int64  
 3   last_scraped                                  3009 non-null   object 
 4   source                                        3009 non-null   object 
 5   name                                          3009 non-null   object 
 6   description                                   3004 non-null   object 
 7   neighborhood_overview                         2290 non-null   object 
 8   picture_url                                   3009 non-null   object 
 9   host_id                                       3009 non-null   i

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [ ]:
#Remove the dollar signs in 'price'
df['price'] = df['price'].replace('$', '')

In [146]:
#some fields are irrelevant for this project, so a few columns were dropped
df_new = df.drop(['scrape_id', 'last_scraped', 'picture_url', 'host_id', 
         'host_url','host_thumbnail_url', 'host_picture_url', 'source'], axis = 1)

In [147]:
#now that the dataset is ready to use, convert it into a sql table
import sqlite3
conn = sqlite3.connect('/df_new.db')
df_new.to_sql('new', conn, if_exists='replace', index=False)

In [148]:
#execute sql queries-testing if the dataset has been converted successfully
conn.execute("""
select id
from new
limit 5
""").fetchall()

[(108061,), (155305,), (156805,), (156926,), (197263,)]

Question 1: What property type is the most expensive property in this dataset? How much does it cost to rent this type of property on average?

In [149]:
#thought process: calculate the average price listing by property types. Round to 2 decimal places. To save some space, only 15 rows are displayed
conn.execute("""
  select property_type, round(avg(price),2) as "average_price"
  from new
  group by property_type 
  order by average_price desc
  limit 15
""").fetchall()

[('Room in boutique hotel', 647.61),
 ('Room in bed and breakfast', 352.21),
 ('Entire chalet', 346.86),
 ('Entire vacation home', 307.75),
 ('Entire villa', 277.5),
 ('Treehouse', 273.33),
 ('Entire townhouse', 268.27),
 ('Entire condo', 251.08),
 ('Shared room in rental unit', 250.0),
 ('Entire loft', 235.28),
 ('Private room in bed and breakfast', 233.35),
 ('Entire cabin', 217.84),
 ('Entire home', 215.19),
 ('Private room in hostel', 214.67),
 ('Room in hotel', 186.5)]

Answer to question 1: Rooms in boutique hotel cost the most on average at a rate of $647.61 

Question 2: what type of property is the most reasonably priced?

In [ ]:
conn.execute("""
  select a.property_type
  from (select property_type, round(avg(price),2) as 'average_price'
        from new
        group by property_type
        order by average_price
        limit 1) as a
""").fetchall()

[('Private room in hut',)]

Question 3: I'd like to look for something immediately availabe and has the highest rating. Is it possible? If so, how much does it cost per night?

In [150]:
#highest rating in 2021
conn.execute("""
  select listing_url, host_name, max(review_scores_rating), price
  from new
  where instant_bookable = 't' and availability_30 > 5
""").fetchall()

[('https://www.airbnb.com/rooms/8070733',
  'Jacqueline & Savannah',
  5.0,
  '150.00')]